In [ ]:
import matplotlib,numpy,glob
import common_functions as cf
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.table import Table
import astropy.units as u
import matplotlib.patches as patches

In [ ]:
font = {'size'   : 14, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1

In [ ]:
HCGs = Table.read('tables/HCGs.vo', format='votable')
HCGs.add_index('HCG')

HCG_mems = Table.read('tables/HCG_members.vo', format='votable')
HCG_mems.add_index('name')

HCGs['logMHI_gals'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_exfs'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_tot'] = numpy.zeros(len(HCGs))+numpy.nan

HCG_mems['logMHI'] = numpy.zeros(len(HCG_mems))+numpy.nan

unresolved_list = [38,49,57,62,89,95]
falsegroup_list = [48]

In [ ]:
for HCG in list(HCGs['HCG']):
    file_list = glob.glob(f'SoFiA_masks/HCG{HCG}/separated_features/*.fits')
    if len(file_list) > 0:
        print(f'Opening the following files for HCG{HCG}:')
        print('\t'+'\n\t'.join(file_list))

        members = list(HCG_mems['name'][HCG_mems['HCG'] == HCG])

        for pb_cubelet in file_list:

            cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(pb_cubelet,True,True)
            
            #Catch the error that SlicerAstro sometimes writes an incorrect value in pixel (0,0,0)
            if abs(cube[0,0,0]) > 1.:
                cube[0,0,0] = 0.

            mhi = numpy.log10(numpy.nansum(cube.flatten()/beam_factor)*abs(cube_dv)*235600.*HCGs.loc[HCG]['dist']**2.)

            #Identify what object is classified as
            group_feature = False
            feat_code = ['br','ex','tl','cl']
            for feature in feat_code:
                if feature in pb_cubelet:
                    inx = pb_cubelet.index(feature)
                    print(f'HCG{HCG}'+pb_cubelet[inx:inx+len(feature)+1]+' is a non-galactic feature.')
                    print(f'HCG{HCG}'+pb_cubelet[inx:inx+len(feature)+1]+f' has a log HI mass of:{mhi}')
                    group_feature = True
                    if numpy.isfinite(HCGs.loc[HCG]['logMHI_exfs']):
                        HCGs.loc[HCG]['logMHI_exfs'] = numpy.log10(10.**mhi + 10.**HCGs.loc[HCG]['logMHI_exfs'])
                    else:
                        HCGs.loc[HCG]['logMHI_exfs'] = mhi
                    break
                        
            group_member = False
            if not group_feature:
                for mem in members:
                    if mem+'.fits' in pb_cubelet:
                        print(f'{mem} is a member of HCG{HCG}.')
                        print(f'{mem} has a log HI mass of:{mhi}')
                        group_member = True
                        HCG_mems.loc[mem]['logMHI'] = mhi
                        if numpy.isfinite(HCGs.loc[HCG]['logMHI_gals']):
                            HCGs.loc[HCG]['logMHI_gals'] = numpy.log10(10.**mhi + 10.**HCGs.loc[HCG]['logMHI_gals'])
                        else:
                            HCGs.loc[HCG]['logMHI_gals'] = mhi
                        break
                        
            #Add exceptions for HCGs 38, 47, 49, 57, 95, and 100
            if ("abcd" in pb_cubelet) or ("acd" in pb_cubelet) or ("ab" in pb_cubelet) or ("ad" in pb_cubelet) or ("abc" in pb_cubelet):
                group_member = True
                inx1 = pb_cubelet.index('features/')+9
                inx2 = pb_cubelet.index('.fits')
                print(pb_cubelet[inx1:inx2]+f' is part of HCG{HCG}.')
                print(pb_cubelet[inx1:inx2]+f' has a log HI mass of:{mhi}')
                if numpy.isfinite(HCGs.loc[HCG]['logMHI_gals']):
                    HCGs.loc[HCG]['logMHI_gals'] = numpy.log10(10.**mhi + 10.**HCGs.loc[HCG]['logMHI_gals'])
                else:
                    HCGs.loc[HCG]['logMHI_gals'] = mhi
                break

            if not group_feature and not group_member:
                inx1 = pb_cubelet.index('features/')+9
                inx2 = pb_cubelet.index('.fits')
                print(pb_cubelet[inx1:inx2]+f' is not part of HCG{HCG}.')
                print(pb_cubelet[inx1:inx2]+f' has a log HI mass of:{mhi}')
                
        if numpy.isfinite(HCGs.loc[HCG]['logMHI_gals']) and numpy.isfinite(HCGs.loc[HCG]['logMHI_exfs']):
            HCGs.loc[HCG]['logMHI_tot'] = numpy.log10(10.**HCGs.loc[HCG]['logMHI_gals'] + 10.**HCGs.loc[HCG]['logMHI_exfs'])
        elif numpy.isfinite(HCGs.loc[HCG]['logMHI_gals']):
            HCGs.loc[HCG]['logMHI_tot'] = HCGs.loc[HCG]['logMHI_gals']
        elif numpy.isfinite(HCGs.loc[HCG]['logMHI_exfs']):
            HCGs.loc[HCG]['logMHI_tot'] = HCGs.loc[HCG]['logMHI_exfs']
        print(f"HCG {HCG} has a log HI mass of:{HCGs.loc[HCG]['logMHI_tot']}")
        
        if HCG in unresolved_list:
            HCGs.loc[HCG]['logMHI_exfs'] = numpy.nan
            HCGs.loc[HCG]['logMHI_gals'] = numpy.nan
        if HCG in falsegroup_list:
            HCGs.loc[HCG]['logMHI_exfs'] = numpy.nan
            HCGs.loc[HCG]['logMHI_gals'] = numpy.nan
    else:
        print(f'No files found for HCG{HCG}.')

    print('\n\n')

In [ ]:
HCGs

In [ ]:
HCGs['ex_frac'] = (numpy.where(numpy.isfinite(HCGs['logMHI_exfs']),10.**HCGs['logMHI_exfs'],0.))/(numpy.where(numpy.isfinite(HCGs['logMHI_gals']),10.**HCGs['logMHI_gals'],0.) + numpy.where(numpy.isfinite(HCGs['logMHI_exfs']),10.**HCGs['logMHI_exfs'],0.))
HCGs['HIdef'] = HCGs['logMHI_pred'] - HCGs['logMHI_tot']

HCGs['logMHI_gals'] = numpy.round(HCGs['logMHI_gals'],decimals=2)
HCGs['logMHI_exfs'] = numpy.round(HCGs['logMHI_exfs'],decimals=2)
HCGs['logMHI_tot'] = numpy.round(HCGs['logMHI_tot'],decimals=2)
HCGs['ex_frac'] = numpy.round(HCGs['ex_frac'],decimals=2)
HCGs['HIdef'] = numpy.round(HCGs['HIdef'],decimals=2)

In [ ]:
HCG_mems['HIdef'] = HCG_mems['logMHI_pred'] - HCG_mems['logMHI']

HCG_mems['logMHI'] = numpy.round(HCG_mems['logMHI'],decimals=2)
HCG_mems['HIdef'] = numpy.round(HCG_mems['HIdef'],decimals=2)

In [ ]:
HCGs

In [ ]:
HCG_mems

In [ ]:
HCGs.write('tables/HCGs_VLA_HI.vo',format='votable',overwrite=True)
HCGs.write('tables/HCGs_VLA_HI.fits',format='fits',overwrite=True)
HCGs.write('tables/HCGs_VLA_HI.ascii',format='ascii.fixed_width_two_line',overwrite=True)

In [ ]:
HCG_mems.write('tables/HCG_mems_VLA_HI.vo',format='votable',overwrite=True)
HCG_mems.write('tables/HCG_mems_VLA_HI.fits',format='fits',overwrite=True)
HCG_mems.write('tables/HCG_mems_VLA_HI.ascii',format='ascii.fixed_width_two_line',overwrite=True)